In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip3 install pandas
!pip3 install datasets

In [28]:
import pandas as pd
from datasets import Dataset, Image, DatasetDict
import argparse


def read_image_file(example):
    with open(example["image"].filename, "rb") as f:
        return {"image": {"bytes": f.read()}}


def get_dataset(root_dir, dataset_name, save_dir, train_split=None, test_split=None, specific_name=None):
    if specific_name:
        path = root_dir + "/" + dataset_name + "/" + specific_name + '.tsv'
    else:
        path = root_dir + "/" + dataset_name + '.tsv'
    if test_split:
        path1 = root_dir + "/" + dataset_name + "/" + train_split + '.tsv'
        path2 = root_dir + "/" + dataset_name + "/" + test_split + '.tsv'
        df1 = pd.read_csv(path1, sep='\t')
        df2 = pd.read_csv(path2, sep='\t')
        df = pd.concat([df1, df2])
    else:
        df = pd.read_csv(path, sep='\t')
    df['file_name'] = root_dir  + "/" + "Images/" + df['file_name']
    df = df.astype(str)
    dataset = Dataset.from_dict({"image": df['file_name'].to_list()}).cast_column("image", Image())
    dataset = dataset.add_column("text", df['text'].to_list())
    dataset = dataset.map(read_image_file)
    train_testvalid = dataset.train_test_split(test_size=0.2, seed=42)
    test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
    train_test_valid_dataset = DatasetDict({
        'train': train_testvalid['train'],
        'test': test_valid['test'],
        'valid': test_valid['train']})
    saved_ds = train_test_valid_dataset.save_to_disk(save_dir)
    return saved_ds

In [29]:
get_dataset("../input/khatt-data/data","OnlineKhatt","./")

In [15]:
!cat ../input/khatt-data/data/OnlineKhatt/Images/OnlineKHATT_A0001_1_1.jpg

In [30]:
!cat ./dataset_dict.json

In [33]:
import datasets

ds = datasets.load_from_disk("./")

In [34]:
ds